**Hi There!** I built some models to predict Legendary Pokemon using various algorithm, 

Please give me some advice to improve my notebook since I'm still a beginner :)

# Read and Inspect the data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/pokemon/Pokemon.csv")

In [ ]:
df.head()

In [ ]:
df.info()

There's no null value here, let's take the quantitative columns (numerical) and then plot their distributions

In [ ]:
numerical = df.iloc[:,4:11]
numerical

In [ ]:
numerical = df.iloc[:,4:11]
fig, axes = plt.subplots(2,4, figsize=(20,10))
a = 0
b = 0
for col in numerical.columns:
    sns.distplot(numerical[col], ax = axes[a][b], kde=False )
    b+=1
    if b == 4:
        b = 0
        a += 1

In [ ]:
sns.countplot(df["Legendary"])

All of the distributions are right-skewed

It's probably due to the number of legendary pokemon is small compared to non-legendary pokemon, and legendary pokemon tends to have higher stats

Now, let's see the distribution of categorical features

In [ ]:
categorical = df[["Type 1", "Type 2", "Generation", "Legendary"]]


fig, axes = plt.subplots(3, figsize=(10,20))
# a = 0
b = 0
for col in categorical.columns:
    if col == "Legendary":
        break
    sns.countplot(y = categorical[col], hue= "Legendary", data = df, ax = axes[b])
#     axes[a][b].set_xticklabels(axes[a][b].get_xticklabels(), rotation=90)
    b+=1
# ax = sns.countplot(df["Type 1"])
# ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

Can't tell much, all of the categorical features seems to be independent

Now, let's inspect the plot some features to see how is the legendary pokemon's stats are dispersed

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x="Total", y="Attack", hue="Legendary", data=df)

In [ ]:
plt.figure(figsize=(8,5))
sns.swarmplot(x="Generation", y="Total", hue="Legendary", data=df)

As what I said before, legendary pokemons tend to have higher stats

# Preprocessing and Prediction

A little preprocessing, so we can see the correlation between features

In [ ]:
from sklearn.preprocessing import LabelEncoder  #For convert categorical feature into number
le = LabelEncoder()
df["Type 1"] = le.fit_transform(df["Type 1"])
df["Type 2"] = df["Type 2"].fillna("Null") #Remove NaN values
df["Type 2"] = le.fit_transform(df["Type 2"])
df.head()

Now let's see the correlations

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(df.corr(), annot = True)

All of the features have big correlations, so I'm going to use them all

Splitting dataset 

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(['Name', 'Legendary'], axis = 1)
y = df['Legendary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0) #Split dataset into train and test set

**Let's predict without further preprocessing**

In [ ]:
result = dict()

Just a simple logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logR = LogisticRegression()
logR.fit(X_train, y_train)
y_pred = logR.predict(X_test)
res = logR.score(X_test, y_test)*100
print(res)

result["logR(NP)"] = res

**88.75%**, not bad at all

But of course it can be better, let's do another preprocessing!


# Logistic Regression

I'm using transformer and pipeline, they're classes provided by scikit-learn for auto preprocessing and modelling so we don't have to write longer codes :D

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import metrics


col_trans = make_column_transformer(
            (OneHotEncoder(),['Type 1', 'Type 2', 'Generation']), #One Hot Encoder, re labelling type columns, because i heard that one hot encoder works better
            (StandardScaler(),['Total','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']), #Scaling numeric data, so the training proccess will be better and faster
            remainder = 'passthrough') #Not doing anything for the rest of columns

Let's try another logistic regression, but with preprocessed data

In [ ]:
pipe = make_pipeline(col_trans,logR)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

res = pipe.score(X_test, y_test)*100
print(res)
result["logR(P)"] = res

As you can see, the accuracy increased 5% because of the preprocessing, **wow!**

# SVM
Now, how about SVM?

In [ ]:
#SVM
from sklearn.svm import SVC

svm_model = SVC()
pipe = make_pipeline(col_trans, svm_model)
pipe.fit(X_train, y_train)
res = pipe.score(X_test, y_test)*100

print(res)
result["SVM"] = res
# y_pred = pipe.predict(X_test)
# print('Accuracy score on Test data: {}'.format(metrics.accuracy_score(y_test,y_pred)*100))

**90.625%**, not better than logistic regression

# Neural Network

F1-Score is a value for calculating Precision and Recall at the same time

In this case, it's just a bonus

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

gamma = 2.0
epsilon = K.epsilon()


Let's just use simple Neural Network, these layers work best so far

In [ ]:

model = Sequential()

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer = Adam(1e-4),
             loss = "binary_crossentropy",
             metrics = ["accuracy", f1_m]
)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
early_stop = EarlyStopping(patience=20, verbose=1, monitor='val_accuracy', mode='max')


In [ ]:
history = model.fit(
        X_train,
        y_train,
        epochs = 100,
        callbacks=[early_stop],
        validation_split=0.2,
        verbose = 0
    
)

In [ ]:
res = model.evaluate(X_train, y_train)[1]*100
result["NN"] = res

**92-94%,** not bad...

# Random Forest and Gradient Boosting

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
pipe = make_pipeline(col_trans, rf)
pipe.fit(X_train, y_train)
res = pipe.score(X_test, y_test)*100

print(res)
result["RF"] = res


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.06, n_estimators = 100,validation_fraction = 0.2, n_iter_no_change=100)
pipe = make_pipeline(col_trans, gb)
pipe.fit(X_train, y_train)
res = pipe.score(X_test, y_test)*100

print(res)
result["GB"] = res

These two work best so far, I will try another algorithm in the future

# Result

In [ ]:
list(result.values())

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
bar_plot = plt.bar(result.keys(), result.values())

def autolabel(rects):
    x = 0
    for idx,rect in enumerate(bar_plot):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 0.5*height,
                round(list(result.values())[x], 2),
                ha='center', va='bottom', rotation=0, size=20, color="white")
        x+=1

autolabel(bar_plot)
ax.set_title("Prediction Result", size=20)

# Conclusion

**That's it!** Gradient Boosting work best for predicting legendary pokemon (so far)

Please kindly give me feedback and advice, so I can improve my notebook, thanks!! :)